In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set geomstats background
import os
os.environ['GEOMSTATS_BACKEND'] = 'numpy'

import numpy as np
import pandas as pd
import scipy.stats as stats
import hyperdt.benchmarking as benchmarking

INFO: Using numpy backend


In [3]:
NUM_POINTS = 1000
NUM_CLASSES = 4
signatures = [
    [(5, -1), (5,- 1)],
    [(5, 1), (5, 1)],
    [(5, -1), (5, 1)],
    [(2, -1), (2, -1), (2, -1), (2, -1), (2, -1)],
    [(2, 1), (2, 1), (2, 1), (2, 1), (2, 1)],
    [(2, -1), (2, -1), (2, 0), (2, 1), (2, 1)]
]

In [4]:
seed = 0
results = []

scores_tuple = benchmarking.compute_scores_by_signature(signatures, NUM_POINTS, NUM_CLASSES, seed=seed, max_depth=3, n_seeds=10)
rnd_seeds = scores_tuple[0]
psdt_scores_by_signature = scores_tuple[1]
psrf_scores_by_signature = scores_tuple[2]
dt_scores_by_signature = scores_tuple[3]
rf_scores_by_signature = scores_tuple[4]
for signature, psdt_score, psrf_score, dt_score, rf_score in zip(signatures, psdt_scores_by_signature,
                                                                       psrf_scores_by_signature, dt_scores_by_signature,
                                                                       rf_scores_by_signature):
    results.append({
        "signature": signature,
        "psdt_score": psdt_score,
        "psrf_score": psrf_score,
        "dt_score": dt_score,
        "rf_score": rf_score
    })

results = pd.DataFrame(results)
results

100%|██████████| 60/60 [07:12<00:00,  7.21s/it]


,signature,psdt_score,psrf_score,dt_score,rf_score
0,"[(5, -1), (5, -1)]","[0.97, 0.98, 0.915, 0.925, 0.98, 1.0, 0.995, 0...","[0.97, 0.98, 0.915, 0.95, 0.985, 1.0, 1.0, 0.9...","[0.95, 0.89, 0.865, 0.835, 0.9, 0.845, 0.985, ...","[0.965, 0.905, 0.885, 0.87, 0.89, 0.855, 0.985..."
1,"[(5, 1), (5, 1)]","[0.675, 0.54, 0.56, 0.6, 0.51, 0.545, 0.67, 0....","[0.675, 0.575, 0.585, 0.62, 0.5, 0.575, 0.675,...","[0.675, 0.535, 0.545, 0.625, 0.535, 0.495, 0.6...","[0.695, 0.56, 0.58, 0.595, 0.545, 0.585, 0.68,..."
2,"[(5, -1), (5, 1)]","[0.97, 0.98, 0.915, 0.925, 0.98, 1.0, 0.995, 0...","[0.97, 0.98, 0.915, 0.95, 0.985, 1.0, 1.0, 0.9...","[0.95, 0.885, 0.865, 0.835, 0.9, 0.87, 0.985, ...","[0.895, 0.92, 0.91, 0.86, 0.87, 0.83, 0.985, 0..."
3,"[(2, -1), (2, -1), (2, -1), (2, -1), (2, -1)]","[0.705, 0.955, 0.72, 0.805, 0.6, 0.66, 0.84, 0...","[0.78, 0.95, 0.725, 0.805, 0.625, 0.735, 0.85,...","[0.72, 0.935, 0.71, 0.805, 0.615, 0.665, 0.8, ...","[0.75, 0.94, 0.72, 0.8, 0.615, 0.68, 0.78, 0.8..."
4,"[(2, 1), (2, 1), (2, 1), (2, 1), (2, 1)]","[0.655, 0.715, 0.49, 0.685, 0.45, 0.575, 0.665...","[0.705, 0.7, 0.51, 0.69, 0.49, 0.615, 0.62, 0....","[0.6, 0.695, 0.495, 0.65, 0.42, 0.6, 0.66, 0.5...","[0.655, 0.735, 0.495, 0.71, 0.455, 0.615, 0.7,..."
5,"[(2, -1), (2, -1), (2, 0), (2, 1), (2, 1)]","[0.77, 0.955, 0.75, 0.805, 0.625, 0.69, 0.84, ...","[0.79, 0.95, 0.735, 0.805, 0.63, 0.74, 0.85, 0...","[0.735, 0.95, 0.735, 0.78, 0.625, 0.65, 0.825,...","[0.755, 0.945, 0.72, 0.815, 0.6, 0.715, 0.795,..."


In [5]:
for signature, (i, row) in zip(signatures, results.iterrows()):
    psdt_scores = row["psdt_score"]
    psrf_scores = row["psrf_score"]
    dt_scores = row["dt_score"]
    rf_scores = row["rf_score"]
    s = []
    for component in signature:
        if component[1] < 0:
            s.append("H(" + f"{component[0]}, {-component[1]}" + ")")
        elif component[1] == 0:
            s.append("E(" + f"{component[0]}" + ")")
        else:
            s.append("S(" + f"{component[0]}, {component[1]}" + ")")
    print(" x ".join(s))
    print(f"Avg PSDT Score: {np.mean(psdt_scores) * 100:.1f} +/- {np.std(psdt_scores) / np.sqrt(len(psdt_scores)) * 1.96 * 100 :.1f}", end=", ")
    print(f"Avg DT Score: {np.mean(dt_scores) * 100:.1f} +/- {np.std(dt_scores) / np.sqrt(len(dt_scores)) * 1.96 * 100 :.1f}")
    
    t, p = stats.ttest_rel(psdt_scores, dt_scores)
    if p < .05:
        print("PSDT and DT are significantly different")
    
    print(f"Avg PSRF Score: {np.mean(psrf_scores) * 100:.1f} +/- {np.std(psrf_scores) / np.sqrt(len(psrf_scores)) * 1.96 * 100 :.1f}", end=", ")
    print(f"Avg RF Score: {np.mean(rf_scores) * 100:.1f} +/- {np.std(rf_scores) / np.sqrt(len(rf_scores)) * 1.96 * 100 :.1f}")

    t, p = stats.ttest_rel(psrf_scores, rf_scores)
    if p < .05:
        print("PSRF and RF are significantly different")
    print("")

H(5, 1) x H(5, 1)
Avg PSDT Score: 96.9 +/- 1.7, Avg DT Score: 91.2 +/- 3.1
PSDT and DT are significantly different
Avg PSRF Score: 97.3 +/- 1.6, Avg RF Score: 92.1 +/- 2.7
PSRF and RF are significantly different

S(5, 1) x S(5, 1)
Avg PSDT Score: 60.2 +/- 3.6, Avg DT Score: 60.3 +/- 4.2
Avg PSRF Score: 62.0 +/- 3.6, Avg RF Score: 61.7 +/- 3.1

H(5, 1) x S(5, 1)
Avg PSDT Score: 96.9 +/- 1.7, Avg DT Score: 91.5 +/- 3.0
PSDT and DT are significantly different
Avg PSRF Score: 97.3 +/- 1.6, Avg RF Score: 91.0 +/- 2.8
PSRF and RF are significantly different

H(2, 1) x H(2, 1) x H(2, 1) x H(2, 1) x H(2, 1)
Avg PSDT Score: 77.0 +/- 6.5, Avg DT Score: 75.4 +/- 5.8
Avg PSRF Score: 78.5 +/- 5.8, Avg RF Score: 76.2 +/- 5.4

S(2, 1) x S(2, 1) x S(2, 1) x S(2, 1) x S(2, 1)
Avg PSDT Score: 57.9 +/- 5.7, Avg DT Score: 56.5 +/- 5.4
Avg PSRF Score: 59.4 +/- 4.9, Avg RF Score: 60.4 +/- 5.6

H(2, 1) x H(2, 1) x E(2) x S(2, 1) x S(2, 1)
Avg PSDT Score: 78.1 +/- 5.7, Avg DT Score: 76.0 +/- 6.0
PSDT and DT a